This notebook has some code for querying wandb runs.

Potential use:
Pull failed runs and collect run commands
for relaunching with `src.slurm.manual_launch`.

### Load libraries

In [1]:
import json
import os
import pathlib
import sys

import git.repo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
import wandb.apis.public
from tqdm.auto import tqdm

GIT_ROOT = pathlib.Path(
    str(git.repo.Repo(".", search_parent_directories=True).working_tree_dir)
)
sys.path.append(str(GIT_ROOT))

from src import utils

### Query wandb data

In [2]:
api = wandb.Api(timeout=30)
runs: list[wandb.apis.public.Run] = api.runs(
    f"data-frugal-learning/finetune",
    filters={
        "$and": [
            {"tags": {"$in": ["finetune-sweep-freeze-v1"]}},
        ]
    },
)

In [3]:
runs = [r for r in runs if r.state not in ("finished", "running")]

In [4]:
for r in runs:
    if "NODE_FAILURE" in r.tags:
        continue

    print(" ".join(utils.get_run_command(r)))

python -m src.pretrain.finetune --dataset_cfg cifar10 --embedder_cfg openai_clip --embedder_cfg.id openai/ViT-B/16 --init_with_trained_linear_probe True --batch_size 50 --seed 0 --n_train 50000 --n_layers_to_freeze 9 --tags finetune-sweep-freeze-v1 --n_val_override 10000
python -m src.pretrain.finetune --dataset_cfg cifar10 --embedder_cfg openai_clip --embedder_cfg.id openai/ViT-B/16 --init_with_trained_linear_probe True --batch_size 50 --seed 0 --n_train 10500 --n_layers_to_freeze 1 --tags finetune-sweep-freeze-v1 --n_val_override 10000
python -m src.pretrain.finetune --dataset_cfg cifar10 --embedder_cfg openai_clip --embedder_cfg.id openai/ViT-B/16 --init_with_trained_linear_probe True --batch_size 50 --seed 0 --n_train 12500 --n_layers_to_freeze 6 --tags finetune-sweep-freeze-v1 --n_val_override 10000
python -m src.pretrain.finetune --dataset_cfg cifar10 --embedder_cfg openai_clip --embedder_cfg.id openai/ViT-B/16 --init_with_trained_linear_probe True --batch_size 50 --seed 0 --n_tr